## Compare Skin Segmentation Between Human Users

In [1]:
import os
import sys
import time
import random
import warnings

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from skimage.io import imread, imshow
from skimage.transform import resize

import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras import backend as K
from keras.utils import multi_gpu_model
from keras import optimizers
from sklearn.metrics import auc # roc curve toolsplt.hold(True)


# Specify Image Dimensions
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed

Using TensorFlow backend.


#### Loading Sub1 and Sub2 Data

In [2]:
MSK_PATH1 = '../forcrosscheck/msk1/'
MSK_PATH2 = '../forcrosscheck/msk2/'


msk_ids1 = next(os.walk(MSK_PATH1))[2]
msk_ids2 = next(os.walk(MSK_PATH2))[2]
msk_ids1.sort()
msk_ids2.sort()

msk1 = np.zeros((len(msk_ids1), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
msk2 = np.zeros((len(msk_ids2), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)

sys.stdout.flush()
for n, id_ in tqdm(enumerate(msk_ids1), total=len(msk_ids1)):
    path = MSK_PATH1 + id_
    img = imread(path)
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    if img.ndim == 3:
        img = img[:,:,1]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True)
    if (np.unique(img).size) > 2:
        img = img > 30           # Important, Needed to make labels 0's and 1's only   
    else:   
        img = img > 0
    img = img.astype(np.uint8)
    msk1[n] = img
    
for n, id_ in tqdm(enumerate(msk_ids2), total=len(msk_ids2)):
    path = MSK_PATH2 + id_
    img = imread(path)
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    img = img.astype(np.uint8)
    msk2[n] = img
    if img.ndim == 3:
        img = img[:,:,1]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True)
    if (np.unique(img).size) > 2:
        img = img > 30           # Important, Needed to make labels 0's and 1's only   
    else:   
        img = img > 0
    img = img.astype(np.uint8)
    msk2[n] = img
    

In [7]:
def acc_comp_anirudh(msk, preds_test_t): #Takes as input 2D masks and 2D predictions
    
    mean_acc = []
    mean_prec = []
    mean_rec = []
    mean_f = []
    for j in range(msk.shape[0]):
        act = msk[j]
        pr = preds_test_t[j].reshape(IMG_WIDTH, IMG_HEIGHT)
    
        c = act == pr #acc
        d = act & pr #true negative
        e = act | pr #true positive
        neg = act.sum()
        pos = (IMG_WIDTH*IMG_HEIGHT)-act.sum()
    
        TN = round(float(d.sum()),6)
        FN = round(float(pr.sum()-d.sum()),6)
        TP = round(float((IMG_WIDTH*IMG_HEIGHT)-e.sum()),6)
        FP = round(float(e.sum()-pr.sum()),6)
        acc = round(float(c.sum())/(IMG_WIDTH*IMG_HEIGHT),6)
    
        acc2 = float(TP+TN)/(TP+TN+FP+FN)
        prec = float(TP)/(TP+FP+0.0000000001)
        rec = float(TP)/(TP+FN)
        f_measure = 2*(prec*rec)/(prec+rec+0.0000000001)
        
        mean_acc.append(acc2)
        mean_prec.append(prec)
        mean_rec.append(rec)
        mean_f.append(f_measure)
    return mean_acc, mean_prec ,mean_rec, mean_f

In [8]:
# Overall accuracy on abdomen pictures
mean_accu, mean_prec,mean_rec, mean_f = acc_comp_anirudh(msk1,msk2);

avg = np.mean(mean_accu)
std = np.std(mean_accu)
precision = np.mean(mean_prec)
recall = np.mean(mean_rec)
fmeasure = np.mean(mean_f)
print("Accuracy " + str(avg))
print("STD Deviation " + str(std))
print("Precision " + str(precision))
print("Recall " + str(recall))
print("F-Measure " + str(fmeasure))

# # Save results in a .npy file
# a = np.reshape(answer[2],(100,1))
# b = np.reshape(answer[3],(100,1))
# c = np.reshape(answer[4],(100,1))
# d = np.reshape(answer[5],(100,1))
# g = np.concatenate([a,b,c,d],axis = 1)
# np.save('your_file_name.npy',g)

Accuracy 0.9679583740234375
STD Deviation 0.0170807022822578
Precision 0.9577195304341507
Recall 0.9376261586575302
F-Measure 0.9466674263949149


In [9]:
# Overall accuracy on abdomen pictures
mean_accu1, mean_prec1,mean_rec1, mean_f1 = acc_comp_anirudh(msk2,msk1);

avg1 = np.mean(mean_accu1)
std1 = np.std(mean_accu1)
precision1 = np.mean(mean_prec1)
recall1 = np.mean(mean_rec1)
fmeasure1 = np.mean(mean_f1)
print("Accuracy " + str(avg1))
print("STD Deviation " + str(std1))
print("Precision " + str(precision1))
print("Recall " + str(recall1))
print("F-Measure " + str(fmeasure1))

Accuracy 0.9679583740234375
STD Deviation 0.0170807022822578
Precision 0.9376261586575092
Recall 0.957719530434173
F-Measure 0.9466674263949147


In [10]:
mean_accu

[0.9666748046875,
 0.9742431640625,
 0.98089599609375,
 0.96429443359375,
 0.9619140625,
 0.95086669921875,
 0.9791259765625,
 0.97650146484375,
 0.97894287109375,
 0.99322509765625,
 0.97772216796875,
 0.95782470703125,
 0.97979736328125,
 0.96783447265625,
 0.9932861328125,
 0.9635009765625,
 0.97650146484375,
 0.9810791015625,
 0.97235107421875,
 0.9530029296875,
 0.974609375,
 0.93182373046875,
 0.96075439453125,
 0.9859619140625,
 0.967041015625,
 0.97796630859375,
 0.96826171875,
 0.9615478515625,
 0.9925537109375,
 0.97564697265625,
 0.98309326171875,
 0.9830322265625,
 0.97344970703125,
 0.9840087890625,
 0.97003173828125,
 0.940673828125,
 0.98089599609375,
 0.95977783203125,
 0.96234130859375,
 0.96240234375,
 0.94970703125,
 0.93231201171875,
 0.95794677734375,
 0.95550537109375,
 0.9947509765625,
 0.97314453125,
 0.96282958984375,
 0.9315185546875,
 0.968017578125,
 0.96832275390625,
 0.9879150390625,
 0.97393798828125,
 0.964599609375,
 0.97589111328125,
 0.96954345703125,